# 背景
互联网实验一般使用基于正态分布模型的检验方法，但是在ab实验中我们可能遇到这样的情况：    

1.实验结果分析，实验组均值比对照组均值提升了10%，相对提升的置信区间是多少呢？
2.实验组用户合计点击率为26%，对照组未25%，置信度与置信区间如何计算？
    
在场景1中，实验组均值、对照组均值是分别服从正态分布的，但是它们的比值会服从正态分布么？标准差怎么计算？    
而场景2中，平均浏览数、平均点击数是服从正态分布的，但平均点击率等于平均点击除以平均浏览。我们又陷入了正态分布随机变量除以正态分布随机变量的问题！    
    
**[Delta method](https://en.wikipedia.org/wiki/Delta_method)可以帮助我们解决这类问题**。    

# Delta method是什么
Delta method说的是当一个随机变量服从正态分布时，经过可导的函数变化后仍然概率趋向正态分布，并且提供了期望、方差的计算公式。  

>单变量下：    
如 $\sqrt{n}[X - \theta] \overset{\nu }{\rightarrow} N(0, \sigma^2)$，且函数g(x)可导，    
则$\sqrt{n}[g(X) - g(\theta)] \overset{\nu }{\rightarrow} N(0, \sigma^2 * [g’(\theta)]^2)$        
    
>多变量下：    
如 $\sqrt{n}[B - \beta] \overset{\nu }{\rightarrow} N(0, \Sigma)$，且函数g(x)可导，    
则$\sqrt{n}[h(B) - h(\theta)] \overset{\nu }{\rightarrow} N(0, \Delta h(B)^T * \Sigma * \Delta h(B))$。    
其中$\Sigma$是多元正态分布的协方差矩阵，$\Delta h$为$h$函数的梯度向量。    
    
# Delta method的直观理解
以下为单变量下的个人理解，不等于严格证明。    
泰勒公式：    
$f(x) = f(a) + \frac{f'(a) }{1!}(x -a)+\frac{f''(a) }{2!}(x -a)^2+...$    
根据泰勒公式：    
$g(X) \approx g(\theta) + g'(\theta)(X - \theta)$    
则：    
$g(X) - g(\theta) \approx  g'(\theta)(X - \theta)  \overset{\nu }{\rightarrow} N(0, \sigma^2 * [g’(\theta)]^2)$    
由于$g'(\theta)(X - \theta)$服从正态分布，左边也近似服从相同的正态分布，且有接近的均值与方差。    
    
# 为什么可以解决AB的问题
场景1与场景2都是两个正态分布随机变量做除法运算的问题，设一个为Xn，一个为Yn，则(Xn, Yn)服从二元正态分布：    
$ (X_n, Y_n) \sim  N((\mu_x，\mu_y), \Sigma)$    
    
我们对Xn,Yn的操作等于函数$h((x, y)) = y/x$  ，根据Delta方法：    
$\frac{Yn}{Xn} \overset{\nu }{\rightarrow}  N(\frac{ E[Yn] }{ E[Xn] }, \Delta h( (X_n, Y_n))^T * \Sigma * \Delta h( (X_n, Y_n)))$    
    
其中$\Delta h((x, y)) = [-\frac{ y}{x^2}, \frac{1}{x}]^T$，$\Sigma = \begin{bmatrix}
{\sigma(X_n)^2 }&{cov(X_n, Y_n)}\\
{cov(X_n, Y_n)}&{\sigma(Y_n)^2}\\
\end{bmatrix}$    

# 联系背景问题
于是我们可以对两个问题的解决方案：    
场景1：$X_n$对照组均值，$Y_n$为实验组均值，使用样本均值、样本方差做期望、方差的点估计；    
场景2：$X_n$为平均用户页面浏览次数，$Y_n$为平均用户页面点击次数，同样使用样本均值、样本方差做期望、方差的点估计。    
    
# 总结
Delta方法对实验分析至关重要，已经几乎成为所有AB实验平台的一部分，主要用来解决随机化单位与分析单位不同的问题。    
Delta方法还可以扩展到更高维度，如微软的CUPED论文中通过四元正态分布的Delta方法解决比例型指标的CUPED计算难点。